# Let's build an ENVIROMENT

Questo è quello che cercheremo di fare in questo notebook: creare un enviroment 2D che sia 
1. **semplice**, intuitivo da usare
2. **versatile**, ergo, in cui si possano posizionare vari oggetti ed agenti così da poter gestire più task
3. **semi-realistico**, cioè che preveda la presenza di leggi fisiche
4. **economico**, in termini di costi computazionali
5. **time-dependent**, cioè che preveda un evoluzione nel tempo 

E quindi? Quindi pensiamo un secondo e mettiamoci al lavoro. 

Useremo sicuramente Pygame per la grafica, è uno strumento semplice e veloce, non si può chiedere di più.

In [35]:
import pygame
import random

Ora, credo che l'enviroment più semplice da pensare sia una "mappa". Questa mappa deve avere la possibilità di gestire vari tipi di terreni e questi terreni devono, in qualche modo, interagire tra loro e con l'agente. 

In [36]:
from abc import ABCMeta, abstractmethod 
class Terrain(metaclass=ABCMeta):
    @abstractmethod
    def __init__(self):
        pass
    #@abstractmethod
    #def time_interation(self):
        pass
    #@abstractmethod
    #def neighbourhood_interaction(self):
        pass
    @abstractmethod
    def agent_interaction(self):
        pass
    @abstractmethod
    def pygame_setup(self):
        pass
    @abstractmethod
    def __str__(self):
        pass

Creeremo una classe astratta che preveda la presenza di tutti i metodi che, qualora qualcuno volesse creare il proprio unico tipo di terreno, devono essere implementati. Noi ci limiteremo a costruire 4 tipi di terreni:
1. **Erba**
2. **Roccia**
3. **Acqua**
4. **Metallo**

Ognuno di essi avrà una propria "fisica" e dunque lasciamo che siano loro a gestire l'aspetto fisico.

Muoversi sull'erba non costa nulla, dunque l'energia dell'agente resterà inalterata. Stare fermi sull'erba fa riguadagnare 1 energia. Interagire con gli oggetti consuma poca energia.

In [37]:
class Grass_terrain(Terrain):
    def __init__(self, **kwargs):
        self.type = "GT"
        self.color = (0, 255, 0)
        self.pos = kwargs['pos']
        self.side = kwargs['side']

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            if agent.action_name == 'stop':
                agent.energy = agent.energy + 1
            if agent.action_name == 'move':
                agent.energy = agent.energy
            if agent.action_name == 'obj_interaction':
                agent.energy = agent.energy - 0.5

    def pygame_setup(self):
        return pygame.Rect(self.pos, (self.side, self.side)), self.color
    
    def __str__(self):
        return f"GRASS TERRAIN"

Poiché la roccia è un terreno più difficile, muoversi su di essa costa molto e costa 2. Stare fermi non farà riguadagnare nulla. Interagire con gli oggetti costa 1.

In [38]:
class Rock_Terrain(Terrain):
    def __init__(self, **kwargs):
        self.type = "RT"
        self.color = (150, 75, 0)
        self.pos = kwargs['pos']
        self.side = kwargs['side']

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            if agent.action_name == 'stop':
                agent.energy = agent.energy 
            if agent.action_name == 'move':
                agent.energy = agent.energy - 2
            if agent.action_name == 'obj_interaction':
                agent.energy = agent.energy - 1

    def pygame_setup(self):
        return pygame.Rect(self.pos, (self.side, self.side)), self.color
    
    def __str__(self):
        return f"ROCK TERRAIN"

L'acqua è un elemento magico. Muoversi è vero, è leggermente più costoso (costa 1.5) tuttavia riposarsi fa recuperare ben 3 energie e interagire con gli oggetti è gratis.

In [39]:
class Water_Terrain(Terrain):
    def __init__(self, **kwargs):
        self.type = "WT"
        self.color = (0, 125, 255)
        self.pos = kwargs['pos']
        self.side = kwargs['side']

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            if agent.action_name == 'stop':
                agent.energy = agent.energy + 3
            if agent.action_name == 'move':
                agent.energy = agent.energy - 1.5
            if agent.action_name == 'obj_interaction':
                agent.energy = agent.energy

    def pygame_setup(self):
        return pygame.Rect(self.pos, (self.side, self.side)), self.color
    
    def __str__(self):
        return f"WATER TERRAIN"

Il metallo è scivoloso, dunque è difficile camminarci sopra. Tuttavia attira calore dunque è buono per riposarsi. Interagire con gli oggetti costa poco.

In [40]:
class Metal_Terrain(Terrain):
    def __init__(self, **kwargs):
        self.type = "MT"
        self.color = (105, 105, 105)
        self.pos = kwargs['pos']
        self.side = kwargs['side']

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            if agent.action_name == 'stop':
                agent.energy = agent.energy + 1.5
            if agent.action_name == 'move':
                agent.energy = agent.energy - 1.5
            if agent.action_name == 'obj_interaction':
                agent.energy = agent.energy - 0.5

    def pygame_setup(self):
        return pygame.Rect(self.pos, (self.side, self.side)), self.color
    
    def __str__(self):
        return f"METAL TERRAIN"

Ora dobbiamo pensare ad una semplice classe per quanto riguarda gli agenti. Ora, decidiamo, in questo momento, ti permettere solo 5 tipi di movimento (dx, sx, up, dw, stop). Lavoreremo su comandi personalizzati per poter compiere altre azioni. Qui pensiamo a costruire una "superclasse" che descrive movimento e simile, tuttavia il "pensiero" è qualcosa di fortemente dipendente da agente ad agente dunque il metodo "choose" di base sarà solo aleatorio.

In [62]:
class Agent:
    def __init__(self, **kwargs):
        self.type = "AA"
        self.pos = kwargs['pos']
        self.color = kwargs['color']
        self.side = kwargs['side']
        self.life = kwargs['life']
        self.reward = kwargs['reward']
        self.object = kwargs['object']
        self.age = kwargs['age']
        self.status = kwargs['status']
        self.energy = kwargs['energy']
        self.name = kwargs['name']
        self.action_name = 'stop' # default
        self.last_action = 'stop' # default
    
    def action(self, act, callback = False):
        self.last_action = act
        if act == 0: # stop
            self.action_name = 'stop'
            self.pos = self.pos
        elif self.energy or callback == True > 0: # the agent need energy to do anything 
            if act == 1: # sx or dw
                self.action_name = 'move'
                self.pos[0] = self.pos[0] - self.side
            if act == 2: # dx or up
                self.action_name = 'move'
                self.pos[0] = self.pos[0] + self.side
            if act == 3: # dw or sx
                self.action_name = 'move'
                self.pos[1] = self.pos[1] - self.side
            if act == 4: # up or dx
                self.action_name = 'move'
                self.pos[1] = self.pos[1] + self.side
            if act == 5: # pickup
                self.action_name = 'obj_interaction'
            if act == 6: # release
                self.action_name = 'obj_interaction'
            if act == 7: # opening door
                self.action_name = 'obj_interaction'
        else:
            self.death()    
    
    def callback(self):
        act = 0
        if self.last_action == 1:
            act = 2
        if self.last_action == 2:
            act = 1
        if self.last_action == 3:
            act = 4
        if self.last_action == 4:
            act = 3
        self.action(act, True)            
        
    def choose(self, playground): # we should pass the playground (or a part of it) to actually perform a choosing process 
        # here is where we implement the actual choosing process
        return random.randint(0, 7)
    
    def death(self):
        self.status = 'dead'
        self.action_name = 'dead'
        self.last_action = 8 # dead
        self.life = 0
    
    def pygame_setup(self):
        return pygame.Rect(self.pos, (self.side, self.side)), self.color

    def __str__(self):
        return f"Agent {self.name.upper()} - type {self.type}"

Aggiungiamo la possibilità di aggiungere oggetti all'interno dell'enviroment. Questo non è un dettaglio da poco. Per ora daremo la possibilità di inserire tre tipi di oggetti:
1. **Muri**
2. **Porte**
3. **Chiavi**

Gli oggetti sono in realtà entità molto semplici che nel nostro caso servono per gestire semplici attività. Devono poter interagire tra di loro (se necessario) e con gli agenti. Nuovamente creeremo una classe astratta per poter dare la possibilità a chi volesse creare in futuro ulteriori oggetti di farlo.

In [42]:
class Object_RL(metaclass=ABCMeta):
    @abstractmethod
    def __init__(self):
        pass
    @abstractmethod
    def agent_interaction(self):
        pass
    @abstractmethod
    def object_interaction(self):
        pass
    @abstractmethod
    def pygame_setup(self):
        pass
    @abstractmethod
    def __str__(self):
        pass

In [43]:
class Wall_object(Object_RL):
    def __init__(self, **kwargs):
        self.type = 'DO'
        self.color = (255, 0, 0)
        self.side = kwargs['side']
        self.pos = kwargs['pos']

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            agent.callback()
    
    def object_interaction(self):
        return super().object_interaction()
    
    def pygame_setup(self):
        return pygame.Rect(self.pos, (self.side, self.side)), self.color

    def __str__(self):
        return f"WALL"

In [44]:
class Door_object(Object_RL):
    def __init__(self, **kwargs):
        self.type = 'DO'
        self.color = (175, 0, 175)
        self.side = kwargs['side']
        self.pos = kwargs['pos']
        self.status = kwargs['status']

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            if self.status == 'close': # quando un agente prova a passare una porta chiusa
                agent.callback()
            else:
                pass
    
    def object_interaction(self, object = None):
        if object == None:
            pass
        else:
            if object.type == 'KO': # quando una chiave prova ad aprire una porta
                if object.action == 1:
                    self.status = 'open' if self.status == 'close' else 'close'
            else:
                pass
    
    def pygame_setup(self):
        if self.status == 'open':
            return pygame.Rect(self.pos, (self.side//3, self.side//3)), self.color
        return pygame.Rect(self.pos, (self.side, self.side)), self.color

    def __str__(self):
        return f"DOOR"

In [45]:
class Key_object(Object_RL):
    def __init__(self, **kwargs):
        self.type = 'KO'
        self.color = (255, 255, 0)
        self.side = kwargs['side']
        self.pos = kwargs['pos']
        self.action = 0 # default
        self.status = 'release' # default
        self.agent = None # default

    def agent_interaction(self, agent = None):
        if agent == None:
            pass
        else:
            if self.agent == agent or self.agent == None:
                if agent.last_action == 7: # quando un agente prova ad aprire una porta
                    self.action = 1
                else:
                    self.action = 0 
                if agent.action == 5: # quando un agente prova a prendere una chiave
                    if agent.object != None:
                        pass
                    if agent.object == None:
                        self.status = 'picked' 
                        self.agent = agent
                        agent.object = self.type
                if agent.action == 6: # quando un agente prova a rilasciare una chiave
                    self.status = 'released'
                    self.agent = None
                    agent.Object = None
            else:
                pass

    
    def object_interaction(self, object = None):
        if object == None:
            pass
        else:
            pass
    
    def pygame_setup(self):
        if self.agent != None:
            self.pos = self.agent.pos
        return pygame.Rect(self.pos, (self.side, self.side)), self.color

    def __str__(self):
        return f"KEY"

Ci resta da unire il tutto in un playground che si evolverà poi col tempo. Questo playground avrà un metodo play() che eseguirà pygame per visualizzarlo. Si può pensare alla possibilità di runnarlo anche senza grafica. Le interazioni avvengono per sovraposizionamento. Questo significa che il playground presenta più layer: il primo per i terreni, il secondo per gli oggetti ed il terzo per gli agenti. Per quanto riguarda il flow delle interazioni, esso deve essere:
1. **Agent - Agent** 
2. **Agent - Object**
3. **Object - Object**
4. **Object - Agent**
5. **Callback** (se ci sono)
6. **Terrain**

In [67]:
class Playground:
    def __init__(self, **kwargs):
        self.width = kwargs['width']
        self.height = kwargs['height']
        self.block_size = kwargs['block_size']
        self.map_terrains = kwargs['terrains']
        self.map_objects = kwargs['objects']
        self.map_agents = kwargs['agents']
        self.fps = kwargs['fps']

    def setup(self):
        pass

    def play(self):

        pygame.init()

        screen = pygame.display.set_mode((self.width, self.height))
        clock = pygame.time.Clock()
        font = pygame.font.SysFont('Arial', 26, bold = True)
        running = True

        while running:
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    pygame.quit()
            
            screen.fill("white")

            for m in self.map_terrains: 
                inf = m.pygame_setup()
                pygame.draw.rect(screen, inf[1], inf[0])
            for o in self.map_objects:
                if o == 0:
                    pass
                inf = o.pygame_setup()
                pygame.draw.rect(screen, inf[1], inf[0])
            for a in self.map_agents:
                if a == 0:
                    pass
                inf = a.pygame_setup()
                pygame.draw.rect(screen, inf[1], inf[0])
            
            for i in range (len(self.map_agents)):
                self.map_agents[i].action(self.map_agents[i].choose(None))
                p = self.map_agents[i].pos
                if p[0] < 0 or p[0] > self.width or p[1] < 0  or p[1] > self.height:
                    self.map_agents[i].callback()
            
            for i in range (len(self.map_objects)):
                po = self.map_objects[i].pos
                for j in range (len(self.map_agents)):
                    pa = self.map_agents[j].pos
                    if po[0] == pa[0] and po[1] == pa[1]:
                        self.map_objects[j].agent_interaction(self.map_agents[j])
            
            for i in range (len(self.map_objects)):
                po = self.map_objects[i].pos
                for j in range (len(self.map_objects)):
                    pa = self.map_objects[j].pos
                    if po[0] == pa[0] and po[1] == pa[1] and i != j:
                        self.map_objects[j].object_interaction(self.map_objects[j])
            
            for i in range (len(self.map_terrains)):
                po = self.map_terrains[i].pos
                for j in range (len(self.map_agents)):
                    pa = self.map_agents[j].pos
                    if po[0] == pa[0] and po[1] == pa[1]:
                        self.map_terrains[j].agent_interaction(self.map_agents[j])
            
            pygame.display.flip()
            clock.tick(self.fps)

    def __str__(self):
        return f"PLAYGROUND"

Fatto ciò non ci resta che provare il nostro enviroment con degli agenti che si comportano in maniera casuale! In un altro notebook proveremo invece a fare qualcosa di più carino. Spoiler : proveremo a fare un gioco in cui gli agenti devono uscire da una stanza in tempo trovando le chiavi. I livelli saranno man mano più difficili.

In [48]:
SIDE = 50

Potremmo pensare a costruire dei file config per queste povere bestie qui. Tuttavia non ora. Dovremo rivedere le logiche di interazione (attualmente creano problemi). Comunque sia fuziona attualmente tutto, dobbiamo rivedere la forma e semplificare le interazioni.

In [68]:
terrains = [Grass_terrain(pos=[i*SIDE, j*SIDE], side=SIDE) if (random.uniform(0,1) > 0.3) else Water_Terrain(pos=[i*SIDE, j*SIDE],side=SIDE) for j in range (SIDE)for i in range (SIDE)]
walls = [Wall_object(side = SIDE, pos = [7*SIDE, j*SIDE]) for j in range (SIDE)]
walls[5] = Door_object(pos=[7*SIDE, 5*SIDE], side = SIDE, status = 'closed')
key = Key_object(pos = [3*SIDE, 3*SIDE], side = SIDE)
objects = []
for w in walls:
    objects.append(w)
objects.append(key)
agent = [Agent(pos = [6*SIDE, 6*SIDE], color = (255, 0, 255),side = SIDE, life = 100, reward = 0, object = None, age = 0, status = 'alive', energy = 100, name = 'John')]
playground = Playground(width = 500, height = 500, block_size = SIDE, terrains = terrains, objects = objects, agents = agent, fps = 3)
playground.play()


error: display Surface quit